# **Title: Image Classification Using Transfer Learning with VGG16**
# 
**Objective**:
The primary goal of this project is to develop a robust image classification model leveraging transfer learning techniques. By utilizing the pre-trained VGG16 architecture, we aim to achieve high accuracy in classifying images into distinct categories. This approach significantly reduces the time required for training and allows us to benefit from the learned features of a model that has already been trained on a vast dataset (ImageNet).

**Background**:
Image classification is a fundamental task in computer vision, where the objective is to assign a label from a predefined set of categories to an input image. Traditional methods require extensive computational resources and large datasets to achieve acceptable performance. However, with the advent of transfer learning, it is now possible to fine-tune existing models on specific tasks with relatively smaller datasets. VGG16, known for its depth and architecture, serves as a powerful base model for transfer learning applications.

**Methodology**:
This project involves the following key steps:

**1. Data Preparation**: Loading and preprocessing the dataset for training and validation.

**2. Model Selection**: Choosing the VGG16 architecture and modifying it to fit our specific classification task.

**3. Training the Model**: Training the model on the prepared dataset while applying data augmentation to enhance generalization.

**4. Evaluation**: Assessing the model’s performance using accuracy metrics and validation loss.


# **1-Data Preparation**

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# 1. Importation des bibliothèques
import os
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator
from PIL import Image

# 2. Définition des chemins et catégories
dataset_path = "/kaggle/input/tunisian-currency"  # Chemin vers votre dataset
categories = ["5dt", "10dt", "20Dt", "50dt"]  # Catégories (noms des dossiers)

# 3. Résumé du dataset
def dataset_summary():
    for category in categories:
        folder_path = os.path.join(dataset_path, category)
        num_images = len(os.listdir(folder_path))
        print(f"{category}: {num_images} images")

# Afficher le nombre d'images par classe
dataset_summary()



In [ ]:
# 4. Visualisation d'échantillons d'images
image_size = (128, 128)

def plot_sample_images():
    plt.figure(figsize=(12, 12))
    for i, category in enumerate(categories):
        folder_path = os.path.join(dataset_path, category)
        image_files = os.listdir(folder_path)[:5]  # Charger les 5 premières images
        for j, image_name in enumerate(image_files):
            img_path = os.path.join(folder_path, image_name)
            img = load_img(img_path, target_size=image_size)
            img_array = img_to_array(img) / 255.0  # Normalisation
            plt.subplot(len(categories), 5, i * 5 + j + 1)
            plt.imshow(img_array)
            plt.axis('off')
            plt.title(category)
    plt.tight_layout()
    plt.show()

# Afficher les échantillons d'images
plot_sample_images()



In [ ]:
# 5. Vérification des tailles d'image
def check_image_sizes():
    for category in categories:
        folder_path = os.path.join(dataset_path, category)
        image_files = os.listdir(folder_path)[:5]  # Charger les 5 premières images
        print(f"Checking image sizes in category {category}:")
        for image_name in image_files:
            img_path = os.path.join(folder_path, image_name)
            img = Image.open(img_path)
            print(f"Image {image_name}: {img.size}")

# Vérifier les dimensions des images
check_image_sizes()



In [ ]:
# 6. Histogrammes de couleur pour une image exemple
def plot_color_histogram(image_path):
    img = load_img(image_path, target_size=image_size)
    img_array = img_to_array(img).astype(np.uint8)

    plt.figure(figsize=(8, 4))
    for i, color in enumerate(["Red", "Green", "Blue"]):
        plt.subplot(1, 3, i + 1)
        plt.hist(img_array[:, :, i].ravel(), bins=256, color=color.lower(), alpha=0.8)
        plt.title(f'{color} Histogram')
    plt.tight_layout()
    plt.show()

# Exemple avec une image
example_image_path = os.path.join(dataset_path, "10dt", os.listdir(os.path.join(dataset_path, "10dt"))[0])
plot_color_histogram(example_image_path)




In [ ]:
# 7. Configuration de l'augmentation d'images
datagen = ImageDataGenerator(
    rotation_range=40,          # Rotation aléatoire jusqu'à 40 degrés
    width_shift_range=0.2,      # Translation horizontale jusqu'à 20% de l'image
    height_shift_range=0.2,     # Translation verticale jusqu'à 20% de l'image
    shear_range=0.2,            # Transformation en cisaillement
    zoom_range=0.2,             # Zoom aléatoire jusqu'à 20%
    horizontal_flip=True,       # Flip horizontal des images
    fill_mode='nearest'         # Remplir les pixels vides après les transformations
)



In [ ]:
# 8. Visualisation d'images augmentées
def augment_and_plot_image(category="10dt", image_index=0):
    # Charger une image de la catégorie
    img_path = os.path.join(dataset_path, category, os.listdir(os.path.join(dataset_path, category))[image_index])
    img = load_img(img_path)  # Charger l'image d'origine
    img_array = img_to_array(img)  # Convertir en tableau numpy

    # Redimensionner l'image pour correspondre aux besoins du modèle (facultatif)
    img_array = img_array.reshape((1,) + img_array.shape)  # Reshape pour correspondre à ImageDataGenerator

    # Créer un plot pour visualiser 5 images augmentées
    plt.figure(figsize=(12, 12))
    i = 0
    for batch in datagen.flow(img_array, batch_size=1):  # Générer des images augmentées
        plt.subplot(3, 3, i + 1)
        plt.imshow(batch[0].astype('uint8'))  # Afficher l'image générée
        plt.axis('off')
        i += 1
        if i % 9 == 0:  # Afficher 9 images
            break
    plt.show()

# Tester l'augmentation sur une image
augment_and_plot_image(category="10dt", image_index=0)


In [ ]:
import os
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Chemin vers votre dataset
dataset_path = "/kaggle/input/tunisian-currency"  # Remplacez par le chemin de votre dataset
categories = ["5dt", "10dt", "20Dt", "50dt"]

# Créer une liste pour stocker les chemins d'images et leurs étiquettes
image_paths = []
labels = []

for category in categories:
    folder_path = os.path.join(dataset_path, category)
    for image_name in os.listdir(folder_path):
        image_paths.append(os.path.join(folder_path, image_name))
        labels.append(category)

# Convertir les étiquettes en tableau NumPy
labels = np.array(labels)

# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(image_paths, labels, test_size=0.2, random_state=42, stratify=labels)

# En outre, vous pouvez diviser l'ensemble d'entraînement en un ensemble de validation
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42, stratify=y_train)

# Afficher le nombre d'images dans chaque ensemble
print(f"Ensemble d'entraînement: {len(X_train)} images")
print(f"Ensemble de validation: {len(X_val)} images")
print(f"Ensemble de test: {len(X_test)} images")


In [ ]:
# Définir un générateur de données avec des augmentations
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,  # Normaliser les images
    rotation_range=40,  # Rotation aléatoire jusqu'à 40 degrés
    width_shift_range=0.2,  # Translation horizontale jusqu'à 20%
    height_shift_range=0.2,  # Translation verticale jusqu'à 20%
    shear_range=0.2,  # Transformation en cisaillement
    zoom_range=0.2,  # Zoom aléatoire jusqu'à 20%
    horizontal_flip=True,  # Flip horizontal des images
    fill_mode='nearest'  # Remplir les pixels vides après les transformations
)

# Générateur pour l'ensemble d'entraînement
train_generator = train_datagen.flow_from_dataframe(
    dataframe=pd.DataFrame({'filename': X_train, 'class': y_train}),
    x_col='filename',
    y_col='class',
    target_size=(128, 128),  # Taille d'image pour le modèle
    batch_size=32,
    class_mode='categorical'  # Utiliser le mode catégorique pour la classification multi-classes
)

# Générateur pour l'ensemble de validation
val_datagen = ImageDataGenerator(rescale=1.0/255.0)  # Normaliser les images
val_generator = val_datagen.flow_from_dataframe(
    dataframe=pd.DataFrame({'filename': X_val, 'class': y_val}),
    x_col='filename',
    y_col='class',
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical'
)

# Générateur pour l'ensemble de test
test_datagen = ImageDataGenerator(rescale=1.0/255.0)  # Normaliser les images
test_generator = test_datagen.flow_from_dataframe(
    dataframe=pd.DataFrame({'filename': X_test, 'class': y_test}),
    x_col='filename',
    y_col='class',
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical'
)


# **2-Model Selection**

In [ ]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model

# Specify the path to the local weights file
weights_path = '/kaggle/input/vgg16-weights/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5'

# Load the model without the classification layers and with local weights
base_model = VGG16(weights=weights_path, include_top=False, input_shape=(128, 128, 3))

# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

# Add custom layers
x = Flatten()(base_model.output)
x = Dense(256, activation='relu')(x)
predictions = Dense(len(categories), activation='softmax')(x)  # Ensure categories is defined

# Create the model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model on your dataset
model.fit(train_generator, validation_data=val_generator, epochs=30)  # Adjust epochs as needed


# **3-Evaluation** 

In [ ]:
# Évaluer le modèle
test_loss, test_accuracy = model.evaluate(test_generator)
print(f'Test accuracy: {test_accuracy:.2f}')


In [ ]:
import matplotlib.pyplot as plt

# Tracer la précision et la perte
plt.figure(figsize=(12, 4))

# Tracer la précision
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

# Tracer la perte
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model

# Chemin vers l'image à tester
test_image_path = '/kaggle/input/testing-data/testing_data/5dt_test.jpg'  # Remplacez par le chemin de votre image

# Charger et préparer l'image
def load_and_preprocess_image(img_path):
    img = image.load_img(img_path, target_size=(128, 128))  # Redimensionner à 128x128
    img_array = image.img_to_array(img)  # Convertir en tableau numpy
    img_array = np.expand_dims(img_array, axis=0)  # Ajouter une dimension pour le batch
    img_array /= 255.0  # Normaliser
    return img_array



# Préparer l'image pour la prédiction
processed_image = load_and_preprocess_image(test_image_path)

# Faire la prédiction
predictions = model.predict(processed_image)
predicted_class = np.argmax(predictions)  # Obtenir la classe prédite

# Mapper la classe à l'étiquette
class_labels = list(train_generator.class_indices.keys())
predicted_label = class_labels[predicted_class]  # Obtenir le nom de la classe prédite

# Afficher l'image avec sa classe prédite
img = image.load_img(test_image_path)
plt.imshow(img)
plt.title(f'Predicted: {predicted_label}')
plt.axis('off')
plt.show()
